In [ ]:
# 1. Proje Kurulumu ve Güncelleme
import os
import sys

# Google Colab kontrolü
try:
    from google.colab import drive
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB:
    print("Google Colab ortamı algılandı.")
    repo_url = "https://github.com/onndd/newproje.git"
    project_dir = "/content/newproje"
    
    if os.path.exists(project_dir):
        print("Proje klasörü mevcut. Güncelleniyor...")
        %cd {project_dir}
        !git pull
    else:
        print("Proje klonlanıyor...")
        !git clone {repo_url}
        %cd {project_dir}
        
    # Gerekli kütüphaneler (Optuna eklendi)
    print("Kütüphaneler yükleniyor...")
    %pip install catboost hmmlearn tensorflow lightgbm scikit-learn optuna
    
    # Path ayarı
    sys.path.append(project_dir)
else:
    print("Yerel ortam algılandı.")
    project_dir = os.getcwd()
    sys.path.append(project_dir)

In [ ]:
# 2. Veri Yükleme ve 3'lü Ayrım (3-Way Split)
import sqlite3
import pandas as pd
import numpy as np
from jetx_project.data_loader import load_data
from jetx_project.config import DB_PATH

print("Veri yükleniyor...")
try:
    df = load_data(DB_PATH)
    print(f"Toplam {len(df)} veri yüklendi.")
    
    # --- 3-WAY SPLIT STRATEJİSİ ---
    # 1. Base Train (%70): Temel modelleri eğitmek için.
    # 2. Meta Train (%15): Temel modellerin tahminlerini alıp Meta-Learner'ı eğitmek için.
    # 3. Test (%15): Final simülasyon ve değerlendirme için (Hiçbir model burayı görmez).
    
    total_len = len(df)
    base_train_end = int(total_len * 0.70)
    meta_train_end = int(total_len * 0.85)
    
    print(f"--- Veri Ayrımı ---")
    print(f"Base Train: 0 - {base_train_end} ({base_train_end} örnek)")
    print(f"Meta Train: {base_train_end} - {meta_train_end} ({meta_train_end - base_train_end} örnek)")
    print(f"Test (Sim): {meta_train_end} - {total_len} ({total_len - meta_train_end} örnek)")
    
except Exception as e:
    print(f"Hata: {e}")
    print("Lütfen jetx.db dosyasının proje klasöründe olduğundan emin olun.")

In [ ]:
# 3. Psikolojik Modeller (HMM) Eğitimi
print("Psikolojik Modeller (HMM) eğitiliyor...")
from jetx_project.model_hmm import train_categorical_hmm, predict_categorical_hmm_states_causal, save_hmm_model

if 'df' in locals() and len(df) > 0:
    # Sadece Base Train verisiyle eğit
    train_values = df['value'].values[:base_train_end]
    
    # HMM Eğitimi (Categorical)
    hmm_model, hmm_map, hmm_bins = train_categorical_hmm(train_values)
    save_hmm_model(hmm_model, hmm_map, bins=hmm_bins, output_dir='models')
    
    # Tüm veri seti için durum tahmini (Feature olarak kullanılacak)
    # CAUSAL PREDICTION: Geleceği görmeden, kayan pencere ile tahmin et.
    all_values = df['value'].values
    hmm_states_mapped = predict_categorical_hmm_states_causal(hmm_model, all_values, hmm_map, bins=hmm_bins, window_size=50)
    
    print("HMM Durumları (Causal) tüm veri için belirlendi.")
    print(f"Durum Dağılımı: {np.bincount(hmm_states_mapped)}")
else:
    print("Veri yok, HMM eğitilemedi.")

In [ ]:
# 4. OPTUNA İLE HİPERPARAMETRE OPTİMİZASYONU (Base Train Üzerinde)
from jetx_project.optimization import optimize_catboost, optimize_lightgbm
from jetx_project.model_a import prepare_model_a_data

if 'df' in locals() and len(df) > 500:
    print("Optimizasyon için veri hazırlanıyor...")
    # Tüm veriyi hazırla ama sadece Base Train kısmını kullan
    X, y_p15, y_p3, y_x = prepare_model_a_data(df['value'].values, hmm_states_mapped)
    
    # Base Train Maskesi
    # X indexleri orijinal df indexleriyle uyumlu olmalı (prepare_model_a_data koruyor mu?)
    # prepare_model_a_data içinde valid_mask var, indexleri resetlemiyor sanırım.
    # Güvenli olmak için iloc kullanalım, ancak X'in uzunluğu len(df) - 1 (shift yüzünden).
    
    # X'in satır sayısı len(df)-1 civarı. 
    # Base Train End indexine kadar olanları alacağız.
    
    # Basitçe: İlk %70'lik dilim.
    limit_idx = base_train_end
    
    # X bir DataFrame, indexleri kontrol edelim
    X_train_opt = X[X.index < limit_idx]
    # y numpy array, maske ile filtrele
    mask_opt = (X.index < limit_idx)
    y_p15_train_opt = y_p15[mask_opt]
    
    print(f"Optimizasyon Veri Seti: {len(X_train_opt)} satır")
    
    print("--- CatBoost Optimizasyonu Başlıyor (GPU) ---")
    best_params_catboost = optimize_catboost(X_train_opt, y_p15_train_opt, n_trials=20)
    print("Bulunan En İyi Parametreler:", best_params_catboost)
else:
    print("Yeterli veri yok.")

In [ ]:
# 5. Model A (CatBoost) Eğitimi (Base Train)
from jetx_project.model_a import train_model_a, save_models
from catboost import CatBoostClassifier

if 'df' in locals() and len(df) > 500:
    print("Model A (CatBoost) eğitiliyor...")
    
    # Sadece Base Train verisini kullan
    limit_idx = base_train_end
    mask_train = (X.index < limit_idx)
    
    X_train_base = X[mask_train]
    y_p15_base = y_p15[mask_train]
    y_p3_base = y_p3[mask_train]
    y_x_base = y_x[mask_train]
    
    # train_model_a kendi içinde %15 validasyon ayırır (Base Train'in %15'i)
    modelA_p15, modelA_p3, modelA_x = train_model_a(X_train_base, y_p15_base, y_p3_base, y_x_base)
    save_models(modelA_p15, modelA_p3, modelA_x, output_dir='models')
    print("Model A tamamlandı.")
else:
    print("Yeterli veri yok.")

In [ ]:
# 6. Model B (k-NN) Eğitimi (Base Train)
from jetx_project.model_b import build_memory, train_model_b, save_memory

if 'df' in locals() and len(df) > 500:
    print("Model B (Hafıza) oluşturuluyor...")
    
    # Sadece Base Train verisiyle hafıza oluştur
    train_values_b = df['value'].values[:base_train_end]
    patterns, targets = build_memory(train_values_b)
    
    nbrs, pca = train_model_b(patterns)
    
    save_memory(nbrs, pca, patterns, targets, output_dir='models')
    print("Model B tamamlandı.")
else:
    print("Yeterli veri yok.")

In [ ]:
# 7. Model C (LSTM) Eğitimi (Base Train)
from jetx_project.model_lstm import train_model_lstm, save_lstm_models

if 'df' in locals() and len(df) > 500:
    print("Model C (LSTM) eğitiliyor...")
    
    # Sadece Base Train verisi
    train_values_c = df['value'].values[:base_train_end]
    
    modelC_p15, modelC_p3, scaler_lstm = train_model_lstm(train_values_c)
    save_lstm_models(modelC_p15, modelC_p3, scaler_lstm, output_dir='models')
    print("Model C tamamlandı.")
else:
    print("Yeterli veri yok.")

In [ ]:
# 8. Model D (LightGBM) Eğitimi (Base Train)
from jetx_project.model_lightgbm import train_model_lightgbm, save_lightgbm_models

if 'df' in locals() and len(df) > 500:
    print("Model D (LightGBM) eğitiliyor...")
    modelD_p15, modelD_p3 = train_model_lightgbm(X_train_base, y_p15_base, y_p3_base)
    save_lightgbm_models(modelD_p15, modelD_p3, output_dir='models')
    print("Model D tamamlandı.")
else:
    print("Yeterli veri yok.")

In [ ]:
# 9. Model E (MLP) Eğitimi (Base Train)
from jetx_project.model_mlp import train_model_mlp, save_mlp_models

if 'df' in locals() and len(df) > 500:
    print("Model E (MLP) eğitiliyor...")
    modelE_p15, modelE_p3, mlp_cols = train_model_mlp(X_train_base, y_p15_base, y_p3_base)
    save_mlp_models(modelE_p15, modelE_p3, mlp_cols, output_dir='models')
    print("Model E tamamlandı.")
else:
    print("Yeterli veri yok.")

In [ ]:
# 9.5. ENSEMBLE STACKING (Meta-Learner) Eğitimi (Meta Train)
from jetx_project.ensemble import prepare_meta_features, train_meta_learner, save_meta_learner
from jetx_project.model_lstm import create_sequences

if 'df' in locals() and len(df) > 2000:
    print("--- Ensemble Stacking (Meta-Learner) Eğitiliyor ---")
    
    # META TRAIN SETİ (%15'lik dilim: base_train_end -> meta_train_end)
    # Bu veri setini Base Modeller HİÇ GÖRMEDİ. 
    # Bu yüzden burada yapacakları tahminler 'Out-of-Sample' niteliğindedir.
    # Meta-Learner bu tahminlere bakarak gerçek performansı öğrenecek.
    
    meta_start_idx = base_train_end
    meta_end_idx = meta_train_end
    
    print(f"Meta Train Aralığı: {meta_start_idx} - {meta_end_idx}")
    
    # 1. Meta Train Seti İçin Tahminleri Topla
    
    # Model A (CatBoost)
    # prepare_model_a_data tüm veriyi döndürür, dilimleyeceğiz
    # Ancak start_index parametresi ile verimli çekebiliriz
    X_meta, _, _, _ = prepare_model_a_data(df['value'].values, hmm_states_mapped, start_index=meta_start_idx)
    # X_meta indexleri meta_start_idx'ten başlar. meta_end_idx'e kadar keseceğiz.
    X_meta = X_meta[X_meta.index < meta_end_idx]
    
    # Hedefler (Meta Train için)
    # df['value'] üzerinden alalım
    # Hedef: Bir sonraki değer (i+1). 
    # X_meta'nın son satırı (meta_end_idx - 1) -> Hedefi meta_end_idx'teki değer.
    y_meta_true_15 = (df['value'].values[meta_start_idx+1 : meta_end_idx+1] >= 1.5).astype(int)
    
    # Uzunluk Kontrolü
    min_len = min(len(X_meta), len(y_meta_true_15))
    X_meta = X_meta.iloc[:min_len]
    y_meta_true_15 = y_meta_true_15[:min_len]
    
    print(f"Meta Train Örnek Sayısı: {min_len}")
    
    preds_a_meta = modelA_p15.predict_proba(X_meta)[:, 1]
    
    # Model B (k-NN)
    preds_b_meta = []
    from jetx_project.model_b import create_pattern_vector, predict_model_b
    pca_obj = locals().get('pca', None) 
    
    # X_meta indexleri üzerinde dönelim
    for idx in X_meta.index:
        pat = create_pattern_vector(df['value'].values, idx)
        if pat is not None:
            p15, _, _ = predict_model_b(nbrs, pca_obj, targets, pat)
            preds_b_meta.append(p15)
        else:
            preds_b_meta.append(0.5)
    preds_b_meta = np.array(preds_b_meta)
            
    # Model C (LSTM)
    # LSTM için veriyi hazırlayalım. 
    # X_meta.index[0] anındaki tahmin için (X_meta.index[0] - seq_len) kadar geriye gitmeliyiz.
    seq_len = 200
    lstm_start_offset = X_meta.index[0] - seq_len
    # Bitiş: X_meta.index[-1] anı (dahil)
    lstm_end_offset = X_meta.index[-1]
    
    meta_values_extended = df['value'].values[lstm_start_offset : lstm_end_offset + 1]
    meta_values_scaled = scaler_lstm.transform(meta_values_extended.reshape(-1, 1))
    
    X_lstm_meta, _, _, _ = create_sequences(meta_values_scaled, seq_len)
    # X_lstm_meta uzunluğu min_len ile aynı olmalı
    preds_c_meta = modelC_p15.predict(X_lstm_meta).flatten()
    
    # Model D (LightGBM)
    preds_d_meta = modelD_p15.predict_proba(X_meta)[:, 1]
    
    # Model E (MLP)
    X_meta_mlp = X_meta[mlp_cols]
    preds_e_meta = modelE_p15.predict_proba(X_meta_mlp)[:, 1]
    
    # HMM States (Meta Train kısmı)
    # X_meta.index'e karşılık gelen state'ler
    hmm_meta = hmm_states_mapped[X_meta.index]
    
    # 2. Meta-Features Hazırla
    # Hepsinin uzunluğu eşitlenmeli (bazen 1-2 fark olabilir)
    final_len = min(len(preds_a_meta), len(preds_b_meta), len(preds_c_meta), len(preds_d_meta), len(preds_e_meta))
    
    # Raw values for 1.00x frequency calculation
    # X_meta corresponds to indices [meta_start_idx : meta_end_idx]
    # We need to slice it exactly as we sliced the predictions
    meta_values_raw = df['value'].values[meta_start_idx : meta_end_idx]
    meta_values_raw = meta_values_raw[:final_len]
    
    meta_X_train = prepare_meta_features(
        preds_a_meta[:final_len], 
        preds_b_meta[:final_len], 
        preds_c_meta[:final_len], 
        preds_d_meta[:final_len], 
        preds_e_meta[:final_len], 
        hmm_meta[:final_len],
        values=meta_values_raw
    )
    y_meta_train = y_meta_true_15[:final_len]
    
    # 3. Eğit
    meta_model, meta_scaler = train_meta_learner(meta_X_train, y_meta_train)
    save_meta_learner(meta_model, meta_scaler, output_dir='models')
    
    print("Meta-Learner eğitildi ve kaydedildi.")
    print("Katsayılar (Hangi modele güveniyor?):")
    print(f"A (Cat): {meta_model.coef_[0][0]:.3f}, B (KNN): {meta_model.coef_[0][1]:.3f}, C (LSTM): {meta_model.coef_[0][2]:.3f}")
    print(f"D (LGB): {meta_model.coef_[0][3]:.3f}, E (MLP): {meta_model.coef_[0][4]:.3f}")
else:
    print("Yeterli veri yok.")

In [ ]:
# 10. BÜYÜK FİNAL: Simülasyon (Test Seti)
import matplotlib.pyplot as plt
from jetx_project.simulation import run_simulation
from jetx_project.evaluation import detailed_evaluation
from jetx_project.ensemble import predict_meta

if 'df' in locals() and len(df) > 2000:
    print("--- BÜYÜK FİNAL: Simülasyon Başlıyor (Test Seti) ---")
    
    # TEST SETİ (%15'lik dilim: meta_train_end -> Son)
    # Bu veriyi ne Base Modeller ne de Meta-Learner gördü.
    # Gerçek performans testi budur.
    
    test_start_idx = meta_train_end
    test_df = df.iloc[test_start_idx:].copy()
    
    print(f"Test Seti Başlangıç: {test_start_idx}, Uzunluk: {len(test_df)}")
    
    # --- Model A (CatBoost) Tahminleri ---
    print("Model A tahmin ediyor...")
    X_test_a, _, _, _ = prepare_model_a_data(df['value'].values, hmm_states_mapped, start_index=test_start_idx)
    # Simülasyon DF'i X_test_a ile aynı uzunlukta olmalı (son satır hariç)
    sim_df = test_df.iloc[:-1].copy()
    # X_test_a bazen 1 eksik/fazla olabilir, hizalayalım
    min_len = min(len(X_test_a), len(sim_df))
    X_test_a = X_test_a.iloc[:min_len]
    sim_df = sim_df.iloc[:min_len]
    
    sim_df['prob_A_1.5'] = modelA_p15.predict_proba(X_test_a)[:, 1]
    sim_df['prob_A_3.0'] = modelA_p3.predict_proba(X_test_a)[:, 1]
    
    # --- Model C (LSTM) Tahminleri ---
    print("Model C (LSTM) tahmin ediyor...")
    seq_len = 200
    # Test başlangıcından geriye seq_len kadar git
    lstm_start = test_start_idx - seq_len
    test_values_extended = df['value'].values[lstm_start:]
    test_values_scaled = scaler_lstm.transform(test_values_extended.reshape(-1, 1))
    X_lstm, _, _, _ = create_sequences(test_values_scaled, seq_len)
    
    preds_c_p15 = modelC_p15.predict(X_lstm).flatten()
    preds_c_p3 = modelC_p3.predict(X_lstm).flatten()
    
    sim_df['prob_C_1.5'] = preds_c_p15[:min_len]
    sim_df['prob_C_3.0'] = preds_c_p3[:min_len]
    
    # --- Model D (LightGBM) Tahminleri ---
    print("Model D (LightGBM) tahmin ediyor...")
    sim_df['prob_D_1.5'] = modelD_p15.predict_proba(X_test_a)[:, 1]
    sim_df['prob_D_3.0'] = modelD_p3.predict_proba(X_test_a)[:, 1]
    
    # --- Model E (MLP) Tahminleri ---
    print("Model E (MLP) tahmin ediyor...")
    X_test_mlp = X_test_a[mlp_cols]
    sim_df['prob_E_1.5'] = modelE_p15.predict_proba(X_test_mlp)[:, 1]
    sim_df['prob_E_3.0'] = modelE_p3.predict_proba(X_test_mlp)[:, 1]
    
    # --- Model B (k-NN) Tahminleri ---
    print("Model B (k-NN) tahmin ediyor...")
    preds_b_sim = []
    pca_obj = locals().get('pca', None)
    
    # sim_df indexleri orijinal df indexleridir (test_start_idx'ten başlar)
    for idx in sim_df.index:
        pat = create_pattern_vector(df['value'].values, idx)
        if pat is not None:
            p15, _, _ = predict_model_b(nbrs, pca_obj, targets, pat)
            preds_b_sim.append(p15)
        else:
            preds_b_sim.append(0.5)
    sim_df['prob_B_1.5'] = preds_b_sim
    
    # --- ENSEMBLE (Meta-Learner) Tahminleri ---
    print("Ensemble (Meta-Learner) tahmin ediyor...")
    # Fix IndexError: Use slicing instead of index lookup
    # hmm_states_mapped covers the entire dataset (0 to total_len)
    # sim_df corresponds to [test_start_idx : test_start_idx + len(sim_df)]
    hmm_sim = hmm_states_mapped[test_start_idx : test_start_idx + len(sim_df)]
    
    meta_X_sim = prepare_meta_features(
        sim_df['prob_A_1.5'], 
        sim_df['prob_B_1.5'], 
        sim_df['prob_C_1.5'], 
        sim_df['prob_D_1.5'], 
        sim_df['prob_E_1.5'], 
        hmm_sim,
        values=sim_df['value'].values
    )
    
    sim_df['prob_Ensemble_1.5'] = predict_meta(meta_model, meta_scaler, meta_X_sim)
    sim_df['prob_Ensemble_3.0'] = sim_df['prob_A_3.0'] 
    
    # --- DETAYLI DEĞERLENDİRME ---
    print("\n--- DETAYLI PERFORMANS ANALİZİ (TEST SETİ) ---")
    
    y_true_15 = (sim_df['value'] >= 1.5).astype(int)
    
    # Model A
    detailed_evaluation(y_true_15, sim_df['prob_A_1.5'], threshold=0.65, model_name="Model A", target_name="1.5x")
    
    # Ensemble
    detailed_evaluation(y_true_15, sim_df['prob_Ensemble_1.5'], threshold=0.65, model_name="ENSEMBLE (Meta)", target_name="1.5x")
    
    # --- SİMÜLASYON KOŞUSU ---
    
    def run_single_model_sim(name, prob_col_15, prob_col_3):
        temp_df = sim_df.copy()
        temp_df['prob_1.5'] = temp_df[prob_col_15]
        temp_df['prob_3.0'] = temp_df[prob_col_3]
        temp_df['pred_value'] = 0 
        
        print(f"\n>>> {name} Simülasyonu <<<")
        run_simulation(temp_df, model_name=name)
        
    run_single_model_sim("Model A (CatBoost)", 'prob_A_1.5', 'prob_A_3.0')
    run_single_model_sim("ENSEMBLE (Meta-Learner)", 'prob_Ensemble_1.5', 'prob_Ensemble_3.0')
    
    # --- GÜVEN DAĞILIM GRAFİKLERİ ---
    print("\n--- Model Güven Dağılımları Hazırlanıyor ---")
    plt.figure(figsize=(15, 6))
    
    plt.title("1.5x Üstü Olasılık Dağılımı (Modeller Ne Kadar Emin?)")
    plt.hist(sim_df['prob_A_1.5'], bins=50, alpha=0.5, label='Model A (CatBoost)')
    plt.hist(sim_df['prob_Ensemble_1.5'], bins=50, alpha=0.5, label='ENSEMBLE (Meta)')
    plt.axvline(x=0.5, color='k', linestyle='--', label='Kararsızlık Sınırı (0.5)')
    plt.axvline(x=0.65, color='r', linestyle='--', label='Hedef Eşik (0.65)')
    plt.legend()
    plt.show()
    
else:
    print("Simülasyon için yeterli veri yok.")

In [ ]:
# 11. Modelleri İndir
!zip -r models.zip models/
from google.colab import files
files.download('models.zip')